In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 20:28].values
classes = np.reshape(classes, (2000, 700, 8))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 8)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNGRU, Bidirectional, Masking

In [5]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNGRU(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Dense(8, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [6]:
accuq8 = []
precisionsq8 = []
recallsq8 = []
f1q8 = []

In [7]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 8))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 8))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accuq8.append(balanced_accuracy_score(y_test, predicted)) 
    precisionsq8.append(precision_score(y_test, predicted, average = 'weighted'))
    recallsq8.append(recall_score(y_test, predicted, average = 'weighted'))
    f1q8.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [8]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0823 23:51:01.072951  2976 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0823 23:51:01.078912  2976 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0823 23:51:01.079908  2976 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   0    2    4 ... 1994 1997 1999] TEST: [   1    3    7   10   11   15   21   24   25   31   32   34   37   38
   41   44   45   47   51   54   61   62   64   66   70   74   77   80
   88   96   98   99  102  105  111  123  128  130  134  136  137  144
  151  158  160  166  168  181  186  188  189  195  196  199  202  223
  233  239  241  269  273  277  280  293  295  311  312  313  315  316
  317  328  330  333  336  337  340  344  346  351  353  354  356  358
  362  365  371  374  377  379  392  402  412  414  416  417  418  426
  435  446  461  462  466  467  481  482  485  496  499  501  508  516
  532  541  544  549  552  554  561  566  569  579  581  587  602  615
  622  624  628  629  631  632  636  644  654  662  665  670  671  677
  683  691  696  706  712  715  716  737  740  744  747  749  754  762
  771  775  778  781  784  786  791  792  793  802  809  813  830  841
  849  853  856  866  869  878  881  886  893  895  899  903  905  922
  928  929  932  936  938  9

W0823 23:51:02.357494  2976 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 16s 10ms/sample - loss: 0.5771 - acc: 0.1604
Epoch 2/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.5294 - acc: 0.1684
Epoch 3/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.5151 - acc: 0.1759
Epoch 4/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.5108 - acc: 0.1779
Epoch 5/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.5062 - acc: 0.1802
Epoch 6/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.5015 - acc: 0.1818
Epoch 7/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4993 - acc: 0.1824
Epoch 8/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4930 - acc: 0.1855
Epoch 9/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4909 - acc: 0.1864
Epoch 10/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4869 - 

1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2968 - acc: 0.2545
Epoch 82/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2952 - acc: 0.2551
Epoch 83/100
1600/1600 [==============================] - 12s 8ms/sample - loss: 0.2946 - acc: 0.2552
Epoch 84/100
1600/1600 [==============================] - 12s 8ms/sample - loss: 0.2931 - acc: 0.2557
Epoch 85/100
1600/1600 [==============================] - 12s 8ms/sample - loss: 0.2925 - acc: 0.2561
Epoch 86/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2916 - acc: 0.2562
Epoch 87/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2902 - acc: 0.2568
Epoch 88/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2901 - acc: 0.2567
Epoch 89/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2889 - acc: 0.2570
Epoch 90/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2875 - acc: 

...

TRAIN: [   1    2    3 ... 1997 1998 1999] TEST: [   0   13   23   28   36   55   57   60   63   72   78   84   86   87
   95  103  104  109  116  135  138  139  140  141  143  145  152  154
  157  159  167  175  179  182  191  192  200  201  203  207  213  218
  224  227  228  230  231  238  240  246  255  258  260  261  263  274
  282  283  291  297  298  299  304  320  321  323  329  342  348  355
  359  370  376  378  381  388  397  425  427  441  443  445  448  452
  458  463  470  477  479  483  491  502  503  512  517  520  523  527
  531  533  534  535  537  540  542  570  588  593  616  618  640  641
  642  643  646  649  650  652  655  659  660  663  667  679  680  685
  687  688  692  693  694  700  705  718  726  727  728  730  731  741
  742  748  755  756  759  770  773  774  776  783  787  789  794  803
  805  807  816  819  825  826  834  836  846  847  850  858  872  875
  877  882  883  884  888  890  901  902  910  913  915  920  926  931
  941  943  948  953  958  9

1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3219 - acc: 0.2480
Epoch 62/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3207 - acc: 0.2482
Epoch 63/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3195 - acc: 0.2487
Epoch 64/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3175 - acc: 0.2492
Epoch 65/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3165 - acc: 0.2496
Epoch 66/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3155 - acc: 0.2502
Epoch 67/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3134 - acc: 0.2506
Epoch 68/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3120 - acc: 0.2513
Epoch 69/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3101 - acc: 0.2516
Epoch 70/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3085 - acc: 

1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4949 - acc: 0.1891
Epoch 10/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4926 - acc: 0.1906
Epoch 11/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4918 - acc: 0.1904
Epoch 12/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4852 - acc: 0.1939
Epoch 13/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4847 - acc: 0.1944
Epoch 14/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4805 - acc: 0.1956
Epoch 15/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4781 - acc: 0.1966
Epoch 16/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4749 - acc: 0.1982
Epoch 17/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4708 - acc: 0.1997
Epoch 18/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.4637 - acc: 

1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2890 - acc: 0.2616
Epoch 90/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2876 - acc: 0.2618
Epoch 91/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2884 - acc: 0.2617
Epoch 92/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2869 - acc: 0.2623
Epoch 93/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2861 - acc: 0.2625
Epoch 94/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2849 - acc: 0.2627
Epoch 95/100
1600/1600 [==============================] - 12s 8ms/sample - loss: 0.2842 - acc: 0.2632
Epoch 96/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2834 - acc: 0.2634
Epoch 97/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2830 - acc: 0.2637
Epoch 98/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.2824 - acc: 

1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3888 - acc: 0.2297
Epoch 38/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3844 - acc: 0.2313
Epoch 39/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3811 - acc: 0.2323
Epoch 40/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3798 - acc: 0.2330
Epoch 41/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3762 - acc: 0.2340
Epoch 42/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3734 - acc: 0.2352
Epoch 43/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3708 - acc: 0.2356
Epoch 44/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3689 - acc: 0.2364
Epoch 45/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3649 - acc: 0.2377
Epoch 46/100
1600/1600 [==============================] - 13s 8ms/sample - loss: 0.3634 - acc: 

In [9]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_20 (Bidirectio (None, 700, 200)          73200     
_________________________________________________________________
dropout_20 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_21 (Bidirectio (None, 700, 200)          181200    
_________________________________________________________________
dropout_21 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 700, 200)          181200    
_________________________________________________________________
dropout_22 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_23 (Bidirectio (None, 700, 200)         

In [11]:
print('Acurácias total')
print(accuq8)
a = np.array(accuq8)
print(a.mean())
print('Precision total')
print(precisionsq8)
p = np.array(precisionsq8)
print(p.mean())
print('Recalls total')
print(recallsq8)
r = np.array(recallsq8)
print(r.mean())
print('F1 total')
print(f1q8)
f = np.array(f1q8)
print(f.mean())

Acurácias total
[0.38577975627475, 0.40290224713797396, 0.3907555082866264, 0.4014853049446528, 0.40642401271868767]
0.3974693658725382
Precision total
[0.5798417927204089, 0.580257621312462, 0.5819719537995791, 0.5797863720337366, 0.5865986070068487]
0.581691269374607
Recalls total
[0.5918001902594616, 0.5986027626288762, 0.5955181167343291, 0.5935787537184907, 0.6011356980472744]
0.5961271042776863
F1 total
[0.5730373819827018, 0.5802167284207054, 0.5765737073482682, 0.5794886290652267, 0.5853840553218973]
0.5789401004277599
